<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335206/logos/Logo_des_Forschungszentrums_J_C3_BClich_seit_2018_hcliq4.svg"  width=250 align='left' style="margin-top:30px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637333672/logos/ebrains_logo_swnlzv.png" width="280" align='left' style="margin-left:50px; margin-top:25px">
</div>  

<br><br><br><br>
<br>

# Simulation of  dose-response curves of agonists using kinetic values

In this tutorial, we will simulate mathematical model of a signaling pathway to obtain dose-response curves, and consequently, predict the *efficacy (EC$_{50}$)* of drugs. 

This tutorial, along with the others in this series, can be completed in Google Colab. If you'd like to open this notebook in Colab, you can use the following link:

<div style='padding:15px'>
<a href="https://colab.research.google.com/github/rribeiro-sci/SSBtoolkit/blob/main/Tutorials/SSBtoolkit-Tutorial3A.ipynb" target="_blank">
<img alt="Colab" src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335713/badges/colab-badge_hh0uyl.svg" height="25" style="margin:20px">
</a>
</div> 


## Installing the SSB toolkit library

In [ ]:
%%capture
!pip install --quiet ssbtoolkit

### Importing dependencies

In [ ]:
import numpy as np
import pandas as pd
import ssbtoolkit as ssb

If you are using the **Ebrains Platform**, execute the following cell to set up the environment. If you are not using **Ebrains**, skip the following step and proceed with the tutorial.

In [ ]:
import os
os.environ['BNGPATH'] = '/opt/app-root/src/.local/lib/python3.8/site-packages/bionetgen/bng-linux/'

## Loading experimental data

Once the SSBtoolkit environment is set up, we are ready to start the simulation.

We will begin by loading the kinetic data of some ligands of the Adenosine 2A receptor. This data was taken from *[Guo, D. et al., 2017](https://pubs.acs.org/doi/10.1021/acs.chemrev.6b00025)* and can be directly downloaded from the repository with the following command.

In [ ]:
!curl -s -O https://raw.githubusercontent.com/rribeiro-sci/SSBtoolkit/main/data/A2AR_Kinetic_values_example.csv

In [ ]:
data = pd.read_csv('A2AR_Kinetic_values_example.csv')
data

Kinetic parameters, k<sub>on</sub> and k<sub>off</sub>, are commonly measured experimentally at low temperatures to slow down the reaction, making it easier to measure. To adjust these kinetic parameters to room temperature (25 °C), we will use the SSB Toolkit’s built-in function `Utils.KineticTempScale()`. This adjustment is performed using the free energy equation:

$ \Delta G = -RTln(K_d)$ 

$K_d = \frac{k_{off}}{k_{on}}$

In [ ]:
# We will create a new dataframe with the scaled values 
data_scaled = data.copy()
data_scaled['kon (1/(uM*s))'] = data.apply(lambda row: ssb.Utils.KineticTempScale(row['kon (1/(uM*s))'],row['koff (1/s)'], row['T (°C)'], 25, Tu='C')[0], axis=1)
data_scaled['koff (1/s)'] = data.apply(lambda row: ssb.Utils.KineticTempScale(row['kon (1/(uM*s))'],row['koff (1/s)'], row['T (°C)'], 25, Tu='C')[1], axis=1)
data_scaled['T (°C)'] = 25
data_scaled

## Selection of the signaling pathway 

The core of the SSB framework is, naturally, the mathematical models of GPCR signaling pathways.

G-protein sub-families are classified by their $\alpha$ subunits, which are used to identify their corresponding signaling pathways:

* G<sub>s</sub>
* G<sub>i/o</sub>
* G<sub>q/11</sub>
* G<sub>12/13</sub>

📖 For more details, refer to the [The SSB toolkit](https://ssbtoolkit.readthedocs.io/) documentation.

You can manually define the G$\alpha$ pathway you want to work with, or use the SSB Toolkit’s built-in function `Utils.GetGProtein()` to query our internal database of human GPCR pathways using the UNIPROT ID of the receptor. The UNIPROT ID for the human Adenosine A2 receptor is [P29274](https://www.uniprot.org/uniprot/P29274).

In [ ]:
uniprotID = 'P29274'
gprotein=ssb.Utils.GetGProtein(uniprotID)
gprotein

<span style="color:black">⚠️ WARNING: The toolkit is specifically designed for human GPCRs. Querying pathways for GPCRs other than human may be included in future updates. However, if you wish to study a non-human GPCR, you can still use the SSB Toolkit by manually setting the G$\alpha$ pathway.</span>

##  Preparation, Simulation and Analysis

To obtain a dose-response curve from the simulation of signaling pathways, you need to perform individual simulations of the pathway across a series of ligand concentrations, similar to what would be done in a wet lab.

We will define an array of ligand concentrations using a geometric progression. This approach is chosen because it addresses the impact of dilution fractions on the accuracy of experimentally estimated K<sub>d</sub> and EC<sub>50</sub>/IC<sub>50</sub> values. The spacing between adjacent concentration values affects the precision of the measurements, particularly in the linear portion of the curve. By using a geometric progression, we can better simulate experimental conditions where each concentration is a power of 2 of the previous concentration *[Sebaugh, J.L., 2011](https://doi.org/10.1002/pst.426)*

<span style="color:black"> ⚠️ WARNING: The SSB toolkit uses μM as default unit for concentration values.</span>



In [ ]:
# Setting the range of ligand concentration
lig_conc_min = 1E-4 # μM
lig_conc_max = 1E4  # μM
lig_conc_range = np.geomspace(lig_conc_min, lig_conc_max, 20) # 20 concentration values

# Setting receptor concentration
receptor_conc = 1E-3 #μM

# Defining other simulation parameters:
time = 10000  # time of simulation in seconds
nsteps = 1000   # number of time steps

## Integration of ODEs 

Once all the simulation parameters have been defined, we are ready to proceed with the simulations. Simulating a mathematical model of a signaling pathway involves integrating a set of ordinary differential equations (ODEs) over time. Each ODE represents a molecular event within the signaling pathway, and integrating these equations describes how the concentration of species in the model changes over time.

In <b>Tutorial 1</b>, the focus was on using the drug-receptor affinity value (K<sub>d</sub>) to initialize the model. In this tutorial, however, we will simulate ligand-receptor binding using kinetic parameters.

Since we are using kinetic values, we need to set `kinetics=True` in the `Simulation.Activation.SetSimulationParameters()` instance. When kinetics=True is selected, you also need to provide a dictionary of parameters. A list of all parameters that can be adjusted for each pathway can be found [here](https://github.com/rribeiro-sci/SSBtoolkit/tree/main/Reactions_Parameters).

For this tutorial, you only need to include the following kinetic parameters:
* `RL_kon` for the ligand-receptor interaction forward parameter (μM<sup>-1</sup>s<sup>-1</sup>)
* `RL_koff` for the ligand-receptor interaction reverse parameter (s<sup>-1</sup>)

In [ ]:
# Preparing the parameters for each ligand:
kinetic_parameters=[]
for k, v in data_scaled.iterrows():
    kinetic_parameters.append({'RL_kon':v['kon (1/(uM*s))'],'RL_koff':v['koff (1/s)']})

In [ ]:
# Creating a simulation instance
sim = ssb.Simulation.Activation()

In [ ]:
# Setting the simulation parameters
sim.SetSimulationParameters(ligands=data_scaled.cmpd.to_list()[0:2], 
                            pathway=gprotein, 
                            receptor_conc=receptor_conc, 
                            lig_conc_range=lig_conc_range, 
                            ttotal=time, 
                            nsteps=nsteps,  
                            binding_kinetics=True, 
                            binding_kinetic_parameters=kinetic_parameters)

In [ ]:
# Starting the simulation
sim.Run()

At the end of the simulation, the concentration values of the species within the signaling pathway will be saved in the instance.

The response of a signaling pathway is typically represented by the increase or decrease in the concentration of one of the species described by the model. To predict the dose-response curve, we need to first extract the maximum concentration value observed for a particular species from each individual simulation (for each ligand concentration). These maximum values will then be fitted to a logistic regression.

To achieve this, we will use the following analysis function:


In [ ]:
sim.Analysis()

We can now to plot the dose-response curves:

In [ ]:
sim.ShowCurve()

Finnaly, from the dose-response curves we can interpolate the EC<sub>50</sub> values.


In [ ]:
sim.ShowPotency()

💡 The predicted potency values can be exported as a Python dictionary using the function `sim.PotencyToDict()`, or saved to a CSV file using `sim.PotencyToCSV()`. 


## Congratulations! 

Congratulations on completing this tutorial notebook! If you enjoyed working through it and would like to continue exploring the SSB Toolkit, we encourage you to complete the rest of the tutorials in this series.

## Cite Us

If you use or adapt this tutorial for your own research projects please cite us.

```
@article{ribeiro_ssb_2022,
    title={{SSB} toolkit: from molecular structure to subcellular signaling pathways.},
    author={Ribeiro, Rui Pedro and Gossen, Jonas and Rossetti, Giulia and Giorgetti, Alejandro},
    publisher={bioRxiv},
    url={https://www.biorxiv.org/content/10.1101/2022.11.08.515595v1},
    doi={10.1101/2022.11.08.515595},
    year={2022}
}
```


## Acknowledgments

EU Human Brain Project (SGA1 and SGA2): This open source software was developed in part in the Human Brain Project funded from the European Union's Horizon 2020 Framework Programme for Research and Innovation under Specific Grant Agreements No 720270 and No. 78907 (Human Brain Project SGA1 and SGA2).

<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
    <img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1642677502/logos/COFUNDED_EU_j2ktlp.jpg" width="300" align='left' style="margin-left:50px">
</div>  